In [4]:
from statsbombpy import sb
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from app.util.football_data_manager import FootballDataManager

fdm = FootballDataManager()
competitions = sb.competitions(fmt='json')

records = []
for key, val in competitions.items():
        # key: tuple (country, competition, season, gender)
        # val: dict with all fields
        record = {
            'competition_id': val.get('competition_id'),
            'season_id': val.get('season_id'),
            'competition_name': val.get('competition_name'),
            'competition_gender': val.get('competition_gender'),
            'country_name': val.get('country_name'),
            'competition_youth': val.get('competition_youth'),
            'competition_international': val.get('competition_international'),
            'season_name': val.get('season_name'),
            'match_updated': val.get('match_updated'),
            'match_updated_360': val.get('match_updated_360'),
            'match_available_360': val.get('match_available_360'),
            'match_available': val.get('match_available'),
            # Also include tuple key fields for reference
            'key_country': key[0] if len(key) > 0 else None,
            'key_competition': key[1] if len(key) > 1 else None,
            'key_season': key[2] if len(key) > 2 else None,
            'key_gender': key[3] if len(key) > 3 else None,
        }
        records.append(record)

records

C:\Users\USER\anaconda3\envs\football-insights-api\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


[{'competition_id': 9,
  'season_id': 281,
  'competition_name': '1. Bundesliga',
  'competition_gender': 'male',
  'country_name': 'Germany',
  'competition_youth': False,
  'competition_international': False,
  'season_name': '2023/2024',
  'match_updated': '2024-07-15T14:15:54.671676',
  'match_updated_360': '2024-07-15T14:17:00.877356',
  'match_available_360': '2024-07-15T14:17:00.877356',
  'match_available': '2024-07-15T14:15:54.671676',
  'key_country': 'Germany',
  'key_competition': '1. Bundesliga',
  'key_season': '2023/2024',
  'key_gender': 'male'},
 {'competition_id': 9,
  'season_id': 27,
  'competition_name': '1. Bundesliga',
  'competition_gender': 'male',
  'country_name': 'Germany',
  'competition_youth': False,
  'competition_international': False,
  'season_name': '2015/2016',
  'match_updated': '2024-05-19T11:11:14.192381',
  'match_updated_360': None,
  'match_available_360': None,
  'match_available': '2024-05-19T11:11:14.192381',
  'key_country': 'Germany',
  '

In [5]:
matches = sb.matches(competition_id=9, season_id=281, fmt='json')

matches_records = []
for k, m in matches.items():
            record = {
                'match_id': m.get('match_id'),
                'match_date': m.get('match_date'),
                'kick_off': m.get('kick_off'),
                'competition_id': m.get('competition', {}).get('competition_id') if isinstance(m.get('competition'), dict) else m.get('competition_id'),
                'competition_name': m.get('competition', {}).get('competition_name') if isinstance(m.get('competition'), dict) else m.get('competition_name'),
                'country_name': m.get('competition', {}).get('country_name') if isinstance(m.get('competition'), dict) else m.get('country_name'),
                'season_id': m.get('season', {}).get('season_id') if isinstance(m.get('season'), dict) else m.get('season_id'),
                'season_name': m.get('season', {}).get('season_name') if isinstance(m.get('season'), dict) else m.get('season_name'),
                'home_team_id': m.get('home_team', {}).get('home_team_id') if isinstance(m.get('home_team'), dict) else m.get('home_team_id'),
                'home_team_name': m.get('home_team', {}).get('home_team_name') if isinstance(m.get('home_team'), dict) else m.get('home_team'),
                'home_team_gender': m.get('home_team', {}).get('home_team_gender') if isinstance(m.get('home_team'), dict) else None,
                'home_team_group': m.get('home_team', {}).get('home_team_group') if isinstance(m.get('home_team'), dict) else None,
                'home_team_country_id': m.get('home_team', {}).get('country', {}).get('id') if isinstance(m.get('home_team', {}).get('country'), dict) else None,
                'home_team_country_name': m.get('home_team', {}).get('country', {}).get('name') if isinstance(m.get('home_team', {}).get('country'), dict) else None,
                'home_manager_id': m.get('home_team', {}).get('managers', [{}])[0].get('id') if m.get('home_team', {}).get('managers') else None,
                'home_manager_name': m.get('home_team', {}).get('managers', [{}])[0].get('name') if m.get('home_team', {}).get('managers') else None,
                'away_team_id': m.get('away_team', {}).get('away_team_id') if isinstance(m.get('away_team'), dict) else m.get('away_team_id'),
                'away_team_name': m.get('away_team', {}).get('away_team_name') if isinstance(m.get('away_team'), dict) else m.get('away_team'),
                'away_team_gender': m.get('away_team', {}).get('away_team_gender') if isinstance(m.get('away_team'), dict) else None,
                'away_team_group': m.get('away_team', {}).get('away_team_group') if isinstance(m.get('away_team'), dict) else None,
                'away_team_country_id': m.get('away_team', {}).get('country', {}).get('id') if isinstance(m.get('away_team', {}).get('country'), dict) else None,
                'away_team_country_name': m.get('away_team', {}).get('country', {}).get('name') if isinstance(m.get('away_team', {}).get('country'), dict) else None,
                'away_manager_id': m.get('away_team', {}).get('managers', [{}])[0].get('id') if m.get('away_team', {}).get('managers') else None,
                'away_manager_name': m.get('away_team', {}).get('managers', [{}])[0].get('name') if m.get('away_team', {}).get('managers') else None,
                'home_score': m.get('home_score'),
                'away_score': m.get('away_score'),
                'match_status': m.get('match_status'),
                'match_status_360': m.get('match_status_360'),
                'last_updated': m.get('last_updated'),
                'last_updated_360': m.get('last_updated_360'),
                'data_version': m.get('metadata', {}).get('data_version') if isinstance(m.get('metadata'), dict) else None,
                'shot_fidelity_version': m.get('metadata', {}).get('shot_fidelity_version') if isinstance(m.get('metadata'), dict) else None,
                'xy_fidelity_version': m.get('metadata', {}).get('xy_fidelity_version') if isinstance(m.get('metadata'), dict) else None,
                'match_week': m.get('match_week'),
                'competition_stage_id': m.get('competition_stage', {}).get('id') if isinstance(m.get('competition_stage'), dict) else None,
                'competition_stage_name': m.get('competition_stage', {}).get('name') if isinstance(m.get('competition_stage'), dict) else None,
                'stadium_id': m.get('stadium', {}).get('id') if isinstance(m.get('stadium'), dict) else None,
                'stadium_name': m.get('stadium', {}).get('name') if isinstance(m.get('stadium'), dict) else None,
                'stadium_country_id': m.get('stadium', {}).get('country', {}).get('id') if isinstance(m.get('stadium', {}).get('country'), dict) else None,
                'stadium_country_name': m.get('stadium', {}).get('country', {}).get('name') if isinstance(m.get('stadium', {}).get('country'), dict) else None,
                'referee_id': m.get('referee', {}).get('id') if isinstance(m.get('referee'), dict) else None,
                'referee_name': m.get('referee', {}).get('name') if isinstance(m.get('referee'), dict) else None,
                'referee_country_id': m.get('referee', {}).get('country', {}).get('id') if isinstance(m.get('referee', {}).get('country'), dict) else None,
                'referee_country_name': m.get('referee', {}).get('country', {}).get('name') if isinstance(m.get('referee', {}).get('country'), dict) else None,
                # Add more fields as needed
            }
            matches_records.append(record)
matches_records[0]

{'match_id': 3895302,
 'match_date': '2024-04-14',
 'kick_off': '17:30:00.000',
 'competition_id': 9,
 'competition_name': '1. Bundesliga',
 'country_name': 'Germany',
 'season_id': 281,
 'season_name': '2023/2024',
 'home_team_id': 904,
 'home_team_name': 'Bayer Leverkusen',
 'home_team_gender': 'male',
 'home_team_group': None,
 'home_team_country_id': 85,
 'home_team_country_name': 'Germany',
 'home_manager_id': 1000310,
 'home_manager_name': 'Xabier Alonso Olano',
 'away_team_id': 176,
 'away_team_name': 'Werder Bremen',
 'away_team_gender': 'male',
 'away_team_group': None,
 'away_team_country_id': 85,
 'away_team_country_name': 'Germany',
 'away_manager_id': 4057,
 'away_manager_name': 'Ole Werner',
 'home_score': 5,
 'away_score': 0,
 'match_status': 'available',
 'match_status_360': 'available',
 'last_updated': '2024-05-10T16:57:53.017895',
 'last_updated_360': '2024-05-10T17:03:59.613154',
 'data_version': '1.1.0',
 'shot_fidelity_version': '2',
 'xy_fidelity_version': '2',
 

In [6]:
events = sb.events(match_id=3895302, fmt='json')

events_records = []
for key, e in events.items():
    record = {
        'id': e.get('id'),
        'index': e.get('index'),
        'period': e.get('period'),
        'timestamp': e.get('timestamp'),
        'minute': e.get('minute'),
        'second': e.get('second'),
        'type_id': e.get('type', {}).get('id') if isinstance(e.get('type'), dict) else None,
        'type_name': e.get('type', {}).get('name') if isinstance(e.get('type'), dict) else e.get('type'),
        'possession': e.get('possession'),
        'possession_team_id': e.get('possession_team', {}).get('id') if isinstance(e.get('possession_team'), dict) else None,
        'possession_team_name': e.get('possession_team', {}).get('name') if isinstance(e.get('possession_team'), dict) else e.get('possession_team'),
        'play_pattern_id': e.get('play_pattern', {}).get('id') if isinstance(e.get('play_pattern'), dict) else None,
        'play_pattern_name': e.get('play_pattern', {}).get('name') if isinstance(e.get('play_pattern'), dict) else e.get('play_pattern'),
        'team_id': e.get('team', {}).get('id') if isinstance(e.get('team'), dict) else None,
        'team_name': e.get('team', {}).get('name') if isinstance(e.get('team'), dict) else e.get('team'),
        'player_id': e.get('player', {}).get('id') if isinstance(e.get('player'), dict) else None,
        'player_name': e.get('player', {}).get('name') if isinstance(e.get('player'), dict) else e.get('player'),
        'location': e.get('location'),
        'duration': e.get('duration'),
        'match_id': e.get('match_id'),
        # Tactics fields
        'tactics_formation': e.get('tactics', {}).get('formation') if isinstance(e.get('tactics'), dict) else None,
        'tactics_lineup': e.get('tactics', {}).get('lineup') if isinstance(e.get('tactics'), dict) else None,
        # Add more fields as needed, including nested ones
    }
    # Optionally flatten more nested fields (e.g., pass, shot, etc.)
    if 'pass' in e:
        record['pass_end_location'] = e['pass'].get('end_location')
        record['pass_outcome'] = e['pass'].get('outcome', {}).get('name') if isinstance(e['pass'].get('outcome'), dict) else e['pass'].get('outcome')
    if 'shot' in e:
        record['shot_end_location'] = e['shot'].get('end_location')
        record['shot_outcome'] = e['shot'].get('outcome', {}).get('name') if isinstance(e['shot'].get('outcome'), dict) else e['shot'].get('outcome')
        record['shot_statsbomb_xg'] = e['shot'].get('statsbomb_xg')
    # Optionally flatten lineup (store as JSON string for DataFrame compatibility)
    if 'tactics' in e and isinstance(e['tactics'], dict) and 'lineup' in e['tactics']:
        import json as _json
        record['tactics_lineup_json'] = _json.dumps(e['tactics']['lineup'])
    events_records.append(record)
events_records

import pandas as pd

events_df = pd.DataFrame(events_records)

events_df = events_df[events_df['tactics_lineup_json'].notnull()] # Filter out rows without tactics lineup
events_df


,id,index,period,timestamp,minute,second,type_id,type_name,possession,possession_team_id,...,duration,match_id,tactics_formation,tactics_lineup,tactics_lineup_json,pass_end_location,pass_outcome,shot_end_location,shot_outcome,shot_statsbomb_xg
0,794ec549-5288-4d1a-93e8-0fc6d3968784,1,1,00:00:00.000,0,0,35,Starting XI,1,904,...,0.0,3895302,3421.0,"[{'player': {'id': 8667, 'name': 'Lukáš Hrádec...","[{""player"": {""id"": 8667, ""name"": ""Luk\u00e1\u0...",NaN,NaN,NaN,NaN,NaN
1,3a9a2c57-b177-47ed-9447-11f081b0e388,2,1,00:00:00.000,0,0,35,Starting XI,1,904,...,0.0,3895302,352.0,"[{'player': {'id': 23198, 'name': 'Michael Zet...","[{""player"": {""id"": 23198, ""name"": ""Michael Zet...",NaN,NaN,NaN,NaN,NaN
3292,32a65be5-91ac-4743-b4e8-75003bd305ed,3293,2,00:23:47.066,68,47,36,Tactical Shift,107,176,...,0.0,3895302,3421.0,"[{'player': {'id': 8667, 'name': 'Lukáš Hrádec...","[{""player"": {""id"": 8667, ""name"": ""Luk\u00e1\u0...",NaN,NaN,NaN,NaN,NaN
3682,ef6e9049-43e1-45ce-bddb-920b5946cb2c,3683,2,00:31:46.845,76,46,36,Tactical Shift,119,176,...,0.0,3895302,352.0,"[{'player': {'id': 23198, 'name': 'Michael Zet...","[{""player"": {""id"": 23198, ""name"": ""Michael Zet...",NaN,NaN,NaN,NaN,NaN


In [7]:
events_fdm = fdm.get_events_for_team(competition_id=9, season_id=281, team_id=904)

# Define competition_id and season_id explicitly
competition_id = 9
season_id = 281

matches = fdm.get_matches(competition_id, season_id)
all_events = [fdm.get_events(match['match_id']) for _, match in matches.iterrows()]
events = pd.concat(all_events) if all_events else None

if events is not None and not events.empty:
    # Try to extract players from tactics_lineup_json if available
    if 'tactics_lineup' in events.columns and events['tactics_lineup'].notnull().any():
        import json
        lineups = events['tactics_lineup_json'].dropna().unique()
        players = []
        for lineup_json in lineups:
            lineup = json.loads(lineup_json)
            for entry in lineup:
                player = entry.get('player', {})
                position = entry.get('position', {})
                players.append({
                    'player_id': player.get('id'),
                    'player_name': player.get('name'),
                    'position_id': position.get('id'),
                    'position_name': position.get('name'),
                    'jersey_number': entry.get('jersey_number')
                })
        print(players)
        
else:
    print("No events found.")

players_df = pd.DataFrame(players)
players_df

matches

C:\Users\USER\anaconda3\envs\football-insights-api\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


[{'player_id': 8667, 'player_name': 'Lukáš Hrádecký', 'position_id': 1, 'position_name': 'Goalkeeper', 'jersey_number': 1}, {'player_id': 27133, 'player_name': 'Odilon Kossonou', 'position_id': 3, 'position_name': 'Right Center Back', 'jersey_number': 6}, {'player_id': 8221, 'player_name': 'Jonathan Tah', 'position_id': 4, 'position_name': 'Center Back', 'jersey_number': 4}, {'player_id': 30606, 'player_name': 'Edmond Fayçal Tapsoba', 'position_id': 5, 'position_name': 'Left Center Back', 'jersey_number': 12}, {'player_id': 41411, 'player_name': 'Nathan Tella', 'position_id': 7, 'position_name': 'Right Wing Back', 'jersey_number': 19}, {'player_id': 38004, 'player_name': 'Piero Martín Hincapié Reyna', 'position_id': 8, 'position_name': 'Left Wing Back', 'jersey_number': 3}, {'player_id': 3500, 'player_name': 'Granit Xhaka', 'position_id': 9, 'position_name': 'Right Defensive Midfield', 'jersey_number': 34}, {'player_id': 9195, 'player_name': 'Robert Andrich', 'position_id': 11, 'positi

,match_id,match_date,kick_off,competition_id,competition_name,country_name,season_id,season_name,home_team_id,home_team_name,...,competition_stage_id,competition_stage_name,stadium_id,stadium_name,stadium_country_id,stadium_country_name,referee_id,referee_name,referee_country_id,referee_country_name
0,3895302,2024-04-14,17:30:00.000,9,1. Bundesliga,Germany,281,2023/2024,904,Bayer Leverkusen,...,1,Regular Season,377,BayArena,85,Germany,234,Harm Osmers,85,Germany
1,3895292,2024-04-06,15:30:00.000,9,1. Bundesliga,Germany,281,2023/2024,190,Union Berlin,...,1,Regular Season,560,Stadion An der Alten Försterei,85,Germany,235,Benjamin Brand,85,Germany
2,3895333,2024-05-05,18:30:00.000,9,1. Bundesliga,Germany,281,2023/2024,184,Eintracht Frankfurt,...,1,Regular Season,116458,Deutsche Bank Park,85,Germany,237,Christian Dingert,85,Germany
3,3895340,2024-05-12,20:30:00.000,9,1. Bundesliga,Germany,281,2023/2024,868,Bochum,...,1,Regular Season,550,Vonovia Ruhrstadion,85,Germany,235,Benjamin Brand,85,Germany
4,3895348,2024-05-18,16:30:00.000,9,1. Bundesliga,Germany,281,2023/2024,904,Bayer Leverkusen,...,1,Regular Season,377,BayArena,85,Germany,837,Matthias Jöllenbeck,85,Germany
5,3895286,2024-03-30,16:30:00.000,9,1. Bundesliga,Germany,281,2023/2024,904,Bayer Leverkusen,...,1,Regular Season,377,BayArena,85,Germany,231,Deniz Aytekin,85,Germany
6,3895220,2024-02-03,16:30:00.000,9,1. Bundesliga,Germany,281,2023/2024,872,Darmstadt 98,...,1,Regular Season,548,Merck-Stadion am Böllenfalltor,85,Germany,461,Tobias Reichel,85,Germany
7,3895250,2024-02-23,21:30:00.000,9,1. Bundesliga,Germany,281,2023/2024,904,Bayer Leverkusen,...,1,Regular Season,377,BayArena,85,Germany,499,Timo Gerach,85,Germany
8,3895266,2024-03-10,20:30:00.000,9,1. Bundesliga,Germany,281,2023/2024,904,Bayer Leverkusen,...,1,Regular Season,377,BayArena,85,Germany,225,Daniel Siebert,85,Germany
9,3895275,2024-03-17,16:30:00.000,9,1. Bundesliga,Germany,281,2023/2024,183,Freiburg,...,1,Regular Season,1000498,Europa-Park Stadion,85,Germany,234,Harm Osmers,85,Germany


In [8]:
from app.services.metric_calculator import load_xt_model

xt_model = load_xt_model()

# Get xT grid and details from xt_model

grid = xt_model.get('xt_grid')
# If xt_model is a dict, use keys instead of attributes
n_grid_cells_x = xt_model.get('grid_size', (None, None))[0]
n_grid_cells_y = xt_model.get('grid_size', (None, None))[1]
pitch_length = xt_model.get('pitch_length', 120)
pitch_width = xt_model.get('pitch_width', 80)

gris_details = {
    "grid_size": {"x": n_grid_cells_x, "y": n_grid_cells_y},
    "pitch_dimensions": {"x": pitch_length, "y": pitch_width},
    "grid_values": grid.tolist()
}
gris_details

AttributeError: 'ExpectedThreatModel' object has no attribute 'get'